<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/knu_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 라이브러리 설치
!pip install PyPDF2 chromadb sentence-transformers pandas

In [9]:
# 벡터 DB에 한번에 저장 => 거의 자동화 바꿀점 2가지 밖에 없음
import os
import re
import unicodedata
import pandas as pd
from datetime import datetime
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
from chromadb import PersistentClient

# 설정: 처음에 한 번만 설정하고 거의 변경하지 않는 항목
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"

# 타겟 폴더 루트 설정 (PDF 또는 HTML 루트)          ################ 여기 2가지만 바꾸기(나머지 건들지 마시오)
ROOT_FOLDER = "/content/drive/MyDrive/pdf"  # 또는 /html, /pdf          변경
URL_MAPPING_FILENAME = "_pdf_url.xlsx"      # 또는 _html_url, _pdf_url  변경

# 학과별 폴더 순회
department_dirs = [d for d in os.listdir(ROOT_FOLDER) if os.path.isdir(os.path.join(ROOT_FOLDER, d))]

# 학과명 매핑
department_name_map = {
    "software_department": "소프트웨어학과",
    "computer_department": "컴퓨터공학과",
    # 필요시 추가
}

# 카테고리 추출 함수 (경로에서 마지막 디렉토리 이름 사용)
def extract_category(file_path, department_root):
    rel_path = os.path.relpath(file_path, start=department_root)
    parts = rel_path.split(os.sep)
    if len(parts) > 1:
        return parts[0]
    return "기타"

# 텍스트 전처리 함수
def clean_text(text: str) -> str:
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[■◆●※▶▷▲→◇]', '', text)
    return text.strip()

# 텍스트 청크화 함수
def split_chunks(text, chunk_size=500):
    return [text[i:i+chunk_size].strip() for i in range(0, len(text), chunk_size)]

# 파일 읽기 함수 (PDF 및 Markdown)
def read_file(filepath):
    if filepath.endswith('.pdf'):
        reader = PdfReader(filepath)
        text = "".join([page.extract_text() for page in reader.pages if page.extract_text()])
    elif filepath.endswith('.md'):
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read()
    else:
        text = ""
    return text

# 하위 모든 파일 경로 가져오기 함수 (폴더 안에 폴더가 있을 수 있음)
def get_all_files(root_dir):
    all_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if f.endswith(('.pdf', '.md')):
                all_files.append(os.path.join(dirpath, f))
    return all_files

# Chroma DB 연결
client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)
existing_ids = set(collection.get(limit=None)['ids'])

# 임베딩 모델 로딩
model = SentenceTransformer(EMBEDDING_MODEL_NAME)

total_added = 0

# 모든 학과 폴더 순회
for dept_dir in department_dirs:
    folder_path = os.path.join(ROOT_FOLDER, dept_dir)
    department_kor = department_name_map.get(dept_dir, dept_dir)

    # URL 매핑 불러오기
    url_mapping_file = os.path.join(folder_path, dept_dir + URL_MAPPING_FILENAME)
    if not os.path.exists(url_mapping_file):
        print(f"❌ URL 매핑 파일 없음: {url_mapping_file}")
        continue

    url_df = pd.read_excel(url_mapping_file)
    url_mapping = dict(zip(url_df['파일명'], url_df['URL']))

    file_paths = get_all_files(folder_path)

    for file_path in file_paths:
        filename = os.path.basename(file_path)
        base_name = unicodedata.normalize("NFC", os.path.splitext(filename)[0])

        raw_text = read_file(file_path)
        if not raw_text.strip():
            print(f"⚠️ {filename}: 텍스트 없음 → 스킵")
            continue

        cleaned_text = clean_text(raw_text)
        chunks = split_chunks(cleaned_text)
        embeddings = model.encode(chunks).tolist()

        ids = [f"{dept_dir}_{base_name}_chunk_{i}" for i in range(len(chunks))]

        new_chunks, new_embeddings, new_ids, new_metadatas = [], [], [], []

        for chunk, emb, id_ in zip(chunks, embeddings, ids):
            if id_ not in existing_ids:
                new_chunks.append(chunk)
                new_embeddings.append(emb)
                new_ids.append(id_)
                new_metadatas.append({
                    "file_name": base_name,
                    "department": department_kor,
                    "category": extract_category(file_path, folder_path),
                    "type": "pdf" if filename.endswith('.pdf') else "markdown",
                    "url": url_mapping.get(base_name, "출처 URL 미등록"),
                    "date": datetime.now().isoformat()
                })

        if new_chunks:
            collection.add(
                documents=new_chunks,
                embeddings=new_embeddings,
                metadatas=new_metadatas,
                ids=new_ids
            )
            print(f"✅ {filename}: {len(new_chunks)}개 저장 완료")
            total_added += len(new_chunks)
        else:
            print(f"⚠️ {filename}: 중복 청크만 존재 → 스킵")

print(f"\n🎉 총 저장된 신규 청크 수: {total_added}")


✅ 소프트웨어전공 졸업규정.pdf: 4개 저장 완료
✅ SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문.pdf: 20개 저장 완료
✅ 졸업논문계획서-샘플.pdf: 1개 저장 완료

🎉 총 저장된 신규 청크 수: 25


In [10]:
# 전체 저장된 데이터 보기
from chromadb import PersistentClient
import pandas as pd

# 벡터 DB 경로와 컬렉션 이름 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

# 클라이언트 및 컬렉션 연결
client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# 전체 데이터 가져오기
results = collection.get(limit=None)

# pandas DataFrame으로 정리
df = pd.DataFrame({
    "id": results["ids"],
    "document": results["documents"],
    "metadata": results["metadatas"]
})

# 예: document는 너무 길 수 있으니 앞부분만 보여주기 (필요시 제거 가능)
df["document"] = df["document"].apply(lambda x: x[:100] + "..." if len(x) > 100 else x)

# 결과 출력
pd.set_option('display.max_rows', None)  # 모든 행 보기
pd.set_option('display.max_colwidth', None)  # 컬럼 길이 제한 해제
display(df)


,id,document,metadata
0,software_department_소프트웨어전공 졸업규정_chunk_0,- 7 -소프트웨어전공 졸업논문 규정 제1조(목적) 이 내규는 공주대학교 학칙(이하 “학칙”이라 함) 제94조 및 공주대학교 학사운영 규정(이하 “학사규정 ”이라 함) 제12장에 ...,"{'category': '규정자료실', 'date': '2025-03-31T15:15:38.694197', 'department': '소프트웨어학과', 'file_name': '소프트웨어전공 졸업규정', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426'}"
1,software_department_소프트웨어전공 졸업규정_chunk_1,"4 이전까지 발표하고 , 종강전까지 인쇄본을 제출하여야한다 ② 단독연구만 을 인정한다 . ③ 지도교수는 졸업논문의 자료준비 , 작성 및 발표 등에 관한 사항을 지도한다 . ④ 졸업...","{'category': '규정자료실', 'date': '2025-03-31T15:15:38.694229', 'department': '소프트웨어학과', 'file_name': '소프트웨어전공 졸업규정', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426'}"
2,software_department_소프트웨어전공 졸업규정_chunk_2,"참조) 및 어학능력 (표2)에 해당하는 자격 취득점수 합계 3점 이상 취득 점수 자격증명 3점국가기술자격시험 1급(기사), 국가공인 SQL전문가 국제공인자격증 (SCA, MCSE,...","{'category': '규정자료실', 'date': '2025-03-31T15:15:38.694243', 'department': '소프트웨어학과', 'file_name': '소프트웨어전공 졸업규정', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426'}"
3,software_department_소프트웨어전공 졸업규정_chunk_3,"작(지도교수 및 대학원생은 인원수 제외, 학술대회 입상은 불인정)를 인정한다 . ③ 전국규모 학술대회 제출 논문(제 1 저자만 인정) 2건 이상을 인정한다 . 제9조(준용) 이 내...","{'category': '규정자료실', 'date': '2025-03-31T15:15:38.694256', 'department': '소프트웨어학과', 'file_name': '소프트웨어전공 졸업규정', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426'}"
4,software_department_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_0,SW중심대학사업 2025학년도 1학기 산학캡스톤디자인 모집공고 국립공주대학교 SW중심대학사업단에서는 2025년도 1학기 산학캡스톤디자인을 다음 과 같이 공고하오니 많은 관심과 적극...,"{'category': '학과공지', 'date': '2025-03-31T15:15:43.296746', 'department': '소프트웨어학과', 'file_name': 'SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=temp_1741306105027100&rs=/synap/result/bbs/1423'}"
5,software_department_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_1,"당 최대 100만원 이내 타 학과와 융합하는 경우 최대 100만원, 단일 학과로 진행할 경우 최대 50만원으로 차등 지원 상기 지원비는 사업단 운영상황에 따라 변동될 수 있음 ...","{'category': '학과공지', 'date': '2025-03-31T15:15:43.296776', 'department': '소프트웨어학과', 'file_name': 'SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=temp_1741306105027100&rs=/synap/result/bbs/1423'}"
6,software_department_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_2,"X_OTNWTnWfv-Inos/edit?usp=sharing 담당 지도 교수 미배정 주제는 팀장 학과를 기준으로 차후 배정 예정 (각 학과에서 진행, 신청 현황에 따라 변경 될 ...","{'category': '학과공지', 'date': '2025-03-31T15:15:43.296797', 'department': '소프트웨어학과', 'file_name': 'SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=temp_1741306105027100&rs=/synap/result/bbs/1423'}"
7,software_department_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_3,(수)~6. 18.(수) *25. 6. 18.(수)까지 결과보고서 제출25. 3. 26.(수) 예정25. 3. 21.(금) 예정 *25. 3. 20.(목) 위원회 심의 상기 일정...,"{'category': '학과공지', 'date': '2025-03-31T15:15:43.296813', 'department': '소프트웨어학과', 'file_name': 'SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=temp_1741306105027100&rs=/synap/result/bbs/1423'}"
8,software_department_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_4,".ac.kr) 제출 ○ 제출서류 : 붙임1{2025년 산학캡스톤디자인 과제 신청(계획)서} ○ 문 의 처: 우연희 연구원(041-521-9971 ) 주관학과 (소프트웨어학과 , ...","{'category': '학과공지', 'date': '2025-03-31T15:15:43.296828', 'department': '소프트웨어학과', 'file_name': 'SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=temp_1741306105027100&rs=/synap/result/bbs/1423'}"
9,software_department_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_5,내용 및 방법의 일관성 )10 연구계획 창의성 Ÿ기존 기술 대비 창의성이 명확하게 기술되어 있는가 ? 10 사업 적합성연구계획 적절성 Ÿ과제 추진 체계(연구자 간 협력 등) 및 성...,"{'category': '학과공지', 'date': '2025-03-31T15:15:43.296840'

In [8]:
# 완전 초기화 ------ 신중히 사용하기 바람 -------
from chromadb import PersistentClient

PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)

# 컬렉션 삭제
client.delete_collection(name=COLLECTION_NAME)
print("🧨 컬렉션 전체 삭제 완료 (초기화됨)")

# 다시 생성
client.create_collection(name=COLLECTION_NAME)
print("✅ 새 빈 컬렉션 재생성 완료")

🧨 컬렉션 전체 삭제 완료 (초기화됨)
✅ 새 빈 컬렉션 재생성 완료
